# TP6 - Procesamiento de Lenguaje Natural

Imports

In [82]:
import pandas as pd
import scipy.io
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns; sns.set_style("darkgrid")
import scipy
import itertools
import random
import math
import operator
import re
import nltk
from nltk.corpus import wordnet
import xml.etree.ElementTree as et
from lxml import etree
from collections import Counter
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.collocations import *
wnl = WordNetLemmatizer()
sns.plt = plt
# nltk.download()

## 1 - Asociacion de palabras
1.1 Levantar el corpus AP, separando cada noticia como un elemento distinto en un diccionario ( < doc_no > : < text > ).

Librerias necesarias: html5lib, lxml, bs4, re

In [62]:
ap_xml_data = open('data/ap.txt').read()
sent_detector = nltk.data.load('tokenizers/punkt/english.pickle')
tokenizer = nltk.tokenize.TreebankWordTokenizer()

def word_normalize(txt):
    return re.sub('[^a-zA-Z0-9-*]', ' ', txt).lower() #.replace("'", "")

def xml2df(xml_data):
    root = et.XML(xml_data, parser=etree.XMLParser(recover=True)) # element tree
    all_records = []
    for i, child in enumerate(root):
        record = {}
        for subchild in child:
            if subchild.tag=="DOCNO":
                record[subchild.tag] = subchild.text
            elif subchild.tag=="TEXT":
                record[subchild.tag] = beautify(word_normalize(subchild.text))
        all_records.append(record)
    df = pd.DataFrame(all_records)
    df.set_index('DOCNO', inplace=True)
    return df

def split(text):
    sentences = sent_detector.tokenize(text)
    tokens = [tokenizer.tokenize(sent) for sent in sentences]
    return tokens

def get_wordnet_pos(treebank_tag):
    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return None

def pos_tag(tokens):
    pos_tokens = [nltk.pos_tag(token) for token in tokens]
    pos_tokens = [[(word, wnl.lemmatize(word,get_wordnet_pos(pos_tag)), [pos_tag]) 
                  for (word,pos_tag) in filter(lambda (w,t): get_wordnet_pos(t) != None,pos)] 
                  for pos in pos_tokens]
    return pos_tokens

def beautify(txt):
    tokens = split(txt)
    lemma_pos_token = pos_tag(tokens)
    return [w2.encode('utf-8') for sublist in lemma_pos_token for (w1,w2,t) in sublist]

ap_df = xml2df(ap_xml_data)

In [69]:
print "AP cargado."
print "Resumen de los primeros 10 documentos:"
print ap_df.iloc[0:10]

AP cargado.
Resumen de los primeros 10 documentos:
                                                              TEXT
DOCNO                                                             
 AP881218-0003   [16-year-old, student, private, baptist, schoo...
 AP880224-0195   [bechtel, group, inc, offer, sell, oil, israel...
 AP881017-0144   [gunman, take, 74-year-old, woman, hostage, be...
 AP881017-0219   [today, be, saturday, day, be, day, leave, yea...
 AP900117-0022   [cupid, have, new, message, lover, valentine, ...
 AP880405-0167   [reagan, administration, be, weigh, invoke, la...
 AP880825-0239   [more, skin, protected, specie, alligator, be,...
 AP880325-0232   [be, organize, union, boost, single, candidate...
 AP880908-0056   [here, be, summary, development, forest, brush...
 AP881105-0097   [jean-pierre, stirbois, man, extreme-right, na...


1.2 Calcular el tamano del vocabulario.

In [71]:
freq_vocab = dict()
for index, row in ap_df.iterrows():
    for word in row[0]:
        freq_vocab[word] = freq_vocab.get(word, 0) + 1

In [72]:
print "El vocabulario tiene: " + str(len(freq_vocab)) + " palabras."

El vocabulario tiene: 32713 palabras.


1.3 Para las 500 palabras con mas apariciones, calcular el par mas asociado segun la medida presentada.

In [75]:
max_freq_vocab = dict(sorted(freq_vocab.iteritems(), key=operator.itemgetter(1), reverse=True)[:500])
print "Se obtuvieron las " + str(len(max_freq_vocab)) + " palabras con mas apariciones."

Se obtuvieron las 500 palabras con mas apariciones.


Usamos PMI:

Sean X,Y palabras, N la cantidad de palabras de todos los textos, W la ventana de co-ocurrencia:

f(X)=occurs(X). f(X,Y)=occurs(Y despues de X, a distancia <= W)/(W-1).

P(X)=f(X)/N. P(X,Y)=f(X,Y)/N.

I(X,Y)=log2(P(X,Y) / (P(X) x P(Y)))=log2( ( f(X,Y) x N ) / ( f(X) x f(Y) ))

In [116]:
bigram_measures = nltk.collocations.BigramAssocMeasures()
def score_associated_pairs(txts, relevant_words, W):
    finder = BigramCollocationFinder.from_words(
        BigramCollocationFinder._build_new_documents(txts, BigramCollocationFinder.default_ws, pad_right=True),
        window_size=W)
    finder.apply_word_filter(lambda w: w not in relevant_words)
    return finder.score_ngrams(bigram_measures.pmi)
scores = score_associated_pairs(ap_df.iloc[:,0], max_freq_vocab.keys(), 5)

In [118]:
print "10 pares mas asociados:"
print scores[0:10]

10 pares mas asociados:
[(('prime', 'minister'), 7.823789312281093), (('cent', 'cent'), 7.604592846739553), (('p', 'm'), 7.538059636214456), (('eastern', 'europe'), 7.38813903518691), (('speak', 'condition'), 7.1853256867452515), (('michael', 'dukakis'), 6.919373328494423), (('human', 'right'), 6.8998780891007705), (('west', 'german'), 6.862453894810415), (('value', 'index'), 6.844544303191942), (('exchange', 'index'), 6.834136037311808)]


In [120]:
print "Palabra mas asociada a cada una de las 500 mas frecuentes (primeras 50):"
most_associated_pairs = dict()
for ((w1,w2),score) in scores:
    if w1 not in most_associated_pairs and w1!=w2:
        most_associated_pairs[w1] = w2
print most_associated_pairs[]

Palabra mas asociada a cada una de las 500 mas frecuentes (primeras 50):


TypeError: unhashable type

## 2 -  Informacion Lexica
Bajar de Project Gutenberg el libro de Darwin ON THE ORIGIN OF SPECIES.

2.1 Procesar el texto, tokenizando eliminando signos de puntuacion.

2.2 Siguiendo el artıculo de la seccion, calcular la autocorrelacion para estimar la distribucion de la palabra a lo largo del texto.

2.3 Armar una funcion que reciba una lista de tokens, una lista de palabras y un tamano de ventana y devuelva una lista de probabilidades de encontrar la palabra en cada ventana para cada palabra pasada por parametro.

2.4 Calcular la entropıa de la distribucion de palabras seleccionadas para distintos tamanos de ventana

2.5 Generar una version randomizada del texto, y medir la entropia de las palabras randomizadas.

2.6 Distinguir las palabras del texto en artıculos, sustantivos y adjetivos usando un POS-tagger. Verificar si las medidas separan a estos grupos de palabras.

## 3 - Word embeddings, distancia semantica y Word- Net
3.1 Utilizando el test WordSim3531, comparar el rendimiento entre LSA[3] y Word2Vec2 [4].

3.2 Comparar los distintos word embeddings con las medidas definidas en WordNet.